## Logistic Regression Example: Predicting Income Status

### Data Source

The UCI Machine Learning Repository [Census Income Dataset](http://archive.ics.uci.edu/ml/datasets/Census+Income) contains information from a 1994 census in the US. A cleaned version of this dataset, `us_census_income_data_clean.csv`, can be found on our GitHub page [here](https://github.com/akmand/datasets).


### Objective

Our goal is to see if we can predict whether a person makes over \\$50K a year or not using logistic regression for the census dataset.


### Target Feature

Our target feature is `income`, which is a binary feature (high: earns over \\$50k a year, low: earns less than \\$50k a year). 

### Descriptive Features

The descriptive features and their data types are given below. 

- **`workclass`**: nominal categorical
- **`education_num`**: numeric
- **`marital_status`**: nominal categorical
- **`occupation`**: nominal categorical
- **`relationship`**: nominal categorical
- **`race`**: binary (White or other)
- **`gender`**: binary (Male or Female)
- **`capital`**: numeric
- **`hours_per_week`**: numeric
- **`native_country`**: binary (United_States or other).

### Exercise 1

First, import the common modules you will be using. And then read in the `us_census_income_data_clean.csv` dataset and display the top 10 rows.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import io
import requests

# so that we can see all the columns
pd.set_option('display.max_columns', None) 

df_url = 'https://raw.githubusercontent.com/akmand/datasets/master/us_census_income_data_clean.csv'
url_content = requests.get(df_url, verify=False).content
data = pd.read_csv(io.StringIO(url_content.decode('utf-8')))

print(f'Data shape = {data.shape}')
data.head(10)

Data shape = (45222, 12)


,age,workclass,education_num,marital_status,occupation,relationship,race,gender,hours_per_week,native_country,capital,income_status
0,39,state_gov,13,never_married,adm_clerical,not_in_family,white,male,40,united_states,2174,<=50k
1,50,self_emp_not_inc,13,married_civ_spouse,exec_managerial,husband,white,male,13,united_states,0,<=50k
2,38,private,9,divorced,handlers_cleaners,not_in_family,white,male,40,united_states,0,<=50k
3,53,private,7,married_civ_spouse,handlers_cleaners,husband,other,male,40,united_states,0,<=50k
4,28,private,13,married_civ_spouse,prof_specialty,wife,other,female,40,other,0,<=50k
5,37,private,14,married_civ_spouse,exec_managerial,wife,white,female,40,united_states,0,<=50k
6,49,private,5,married_spouse_absent,other_service,not_in_family,other,female,16,other,0,<=50k
7,52,self_emp_not_inc,9,married_civ_spouse,exec_managerial,husband,white,male,45,united_states,0,>50k
8,31,private,14,never_married,prof_specialty,not_in_family,white,female,50,united_states,14084,>50k
9,42,private,13,married_civ_spouse,exec_managerial,husband,white,male,40,united_states,5178,>50k


### Exercise 2

First, clean up the response variable as follows:
```Python
data['income_status'] = data['income_status'].replace({'<=50k': 0, '>50k': 1}).astype(object)
```
Display the unique value counts for all the categorical columns. 

**HINT:** In `Pandas`, string types are of data type "object", and usually these would be the categorical features.

In [2]:
data['income_status'] = data['income_status'].replace({'<=50k': 0, '>50k': 1}).astype(object)

In [3]:
categoricalColumns = data.columns[data.dtypes==object].tolist()

for col in categoricalColumns:
    print('Unique values for ' + col)
    print(data[col].value_counts())
    print('')

Unique values for workclass
private             33307
self_emp_not_inc     3796
local_gov            3100
state_gov            1946
self_emp_inc         1646
federal_gov          1406
without_pay            21
Name: workclass, dtype: int64

Unique values for marital_status
married_civ_spouse       21055
never_married            14598
divorced                  6297
separated                 1411
widowed                   1277
married_spouse_absent      552
married_af_spouse           32
Name: marital_status, dtype: int64

Unique values for occupation
craft_repair         6020
prof_specialty       6008
exec_managerial      5984
adm_clerical         5540
sales                5408
other_service        4808
machine_op_inspct    2970
transport_moving     2316
handlers_cleaners    2046
farming_fishing      1480
tech_support         1420
protective_serv       976
priv_house_serv       232
armed_forces           14
Name: occupation, dtype: int64

Unique values for relationship
husband          

## Exercise 3

Construct the logistic regression formula as a Python string. Also add the square of the `hours_per_week` feature to illustrate how you can add higher order terms to logistic regression.

**HINT:** When constructing the logistic regression formula, you can manually add all the independent features. On the other hand, if there are lots of independent variables, you can get smart and use the `join()` string function.

In [4]:
formula_string_indep_vars = ' + '.join(data.drop(columns='income_status').columns)
formula_string = 'income_status ~ ' + formula_string_indep_vars
print('formula_string: ', formula_string)

formula_string:  income_status ~ age + workclass + education_num + marital_status + occupation + relationship + race + gender + hours_per_week + native_country + capital


In [5]:
formula_string = formula_string + ' + np.power(hours_per_week, 2)'
print('formula_string: ', formula_string)

formula_string:  income_status ~ age + workclass + education_num + marital_status + occupation + relationship + race + gender + hours_per_week + native_country + capital + np.power(hours_per_week, 2)


### Exercise 4

Now that you have defined the statistical model formula as a Python string, fit an *logistic regression model* to the data.

In [6]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

model_full = smf.glm(formula=formula_string, data=data, family=sm.families.Binomial())
###
model_full_fitted = model_full.fit()
###
print(model_full_fitted.summary())

                            Generalized Linear Model Regression Results                             
Dep. Variable:     ['income_status[0]', 'income_status[1]']   No. Observations:                45222
Model:                                                  GLM   Df Residuals:                    45183
Model Family:                                      Binomial   Df Model:                           38
Link Function:                                        Logit   Scale:                          1.0000
Method:                                                IRLS   Log-Likelihood:                -15081.
Date:                                      Sat, 26 Feb 2022   Deviance:                       30161.
Time:                                              21:10:10   Pearson chi2:                 7.75e+04
No. Iterations:                                           8   Pseudo R-squ. (CS):             0.3642
Covariance Type:                                  nonrobust                                

### Exercise 5
Define a new data frame for actual income vs. predicted income and display the top 10 rows.

In [7]:
residuals_full = pd.DataFrame({'actual': data['income_status'], 
                            'predicted_proba': model_full_fitted.fittedvalues, 
                            'predicted_income': np.where(np.round(model_full_fitted.fittedvalues) < 0.5, 
                                                         '1', 
                                                         '0')})
residuals_full.head(10)

,actual,predicted_proba,predicted_income
0,0,0.862740,0
1,0,0.718657,0
2,0,0.971674,0
3,0,0.888291,0
4,0,0.365294,1
5,0,0.167451,1
6,0,0.998926,0
7,1,0.537058,0
8,1,0.157606,1
9,1,0.096465,1


### Exercise 6

Consider an individual with the attributes below:
- `age` = 40
- `workclass` = private
- `education_num` = 15
- `marital_status` = married_civ_spouse
- `occupation` = prof_specialty
- `relationship`: husband
- `race`: white
- `gender`: male
- `hours_per_week`: 40
- `native_country`: united_states
- `capital`: 10000

Calculate the probability of this individual being a high income person. What is the prediction of the logistic regression model in this particular case?

In [8]:
new_obs = pd.DataFrame({
    'age': [40],
    'workclass': ['private'],
    'education_num': [15],
    'marital_status': ['married_civ_spouse'],
    'occupation': ['prof_specialty'],
    'relationship': ['husband'],
    'race': ['white'],
    'gender': ['male'],
    'hours_per_week': [40],
    'native_country': ['united_states'],
    'capital': [10000],
})
new_obs

,age,workclass,education_num,marital_status,occupation,relationship,race,gender,hours_per_week,native_country,capital
0,40,private,15,married_civ_spouse,prof_specialty,husband,white,male,40,united_states,10000


In [9]:
model_full_fitted.predict(pd.DataFrame(new_obs))

0    0.023215
dtype: float64

The predicted probability for class `0`, which is `<50k` (low income), is about 2%. That is, the model predicts a 98% probability for high income class for this individual.

***
www.featureranking.com